In [13]:
import pandas as pd
import awswrangler as wr
import boto3
from dotenv import load_dotenv
import os

In [14]:
load_dotenv()

True

In [15]:
aws_access_key_id = os.getenv("aws_access_key_id")
aws_secret_access_key = os.getenv("aws_secret_access_key")

In [26]:
raw_database = 'raw-anakin'
database_name = 'silver-darthvader'
table_name = 'people' # laço de uma lista
layer = 'clean'
path = 's3://silver-clean-darthvader/planets'
file = f"/Users/arthur/DeathStarLake/transformations/{table_name}.sql"

In [27]:
session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key, 
                                region_name='us-east-2')

In [28]:
wr.catalog.create_database(silver_base, exist_ok=True, boto3_session=session)

In [29]:
with open(file) as f:  
    query = f.read()

In [31]:
data = wr.athena.read_sql_query(sql=query, database=raw_database, boto3_session=session)

In [33]:
wr.s3.to_parquet(
    df=data,
    path=path,
    dataset=True,
    mode="overwrite",
    database=database_name,
    table=table_name,
    boto3_session=session
)

{'paths': ['s3://silver-clean-darthvader/planets/026b1dd693674676be6435434bad4878.snappy.parquet'],
 'partitions_values': {}}